<a href="https://colab.research.google.com/github/vblagoje/notebooks/blob/main/haystack2x-demos/haystack_rag_services_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

This notebook showcases a novel application of Qdrant vector DB in the upcoming Haystack OpenAPI service-based Retriever-Augmented Generation (RAG) system. Given a user query, we retrieve the corresponding OpenAPI specification from Qdrant vector DB. The retrieved service specification is then seamlessly injected into the Haystack pipeline, enabling the dynamic invocation of selected API services. This innovative approach, integrating any OpenAPI spec service with LLMs, opens a new era for RAG systems to generate contextually rich responses from any openapi service.

## 1. Setup
This notebook demos Haystack 2.x service based RAG, using two openapi services: GitHub's compare_branches and SerperDev search API.

Let's install necessary libraries and import key modules to build the foundation for the subsequent steps.

In [ ]:
!pip uninstall -y llmx

Found existing installation: llmx 0.0.15a0
Uninstalling llmx-0.0.15a0:
  Successfully uninstalled llmx-0.0.15a0


In [ ]:
!pip install -q "grpcio-tools==1.42" sentence-transformers openapi3 jsonref

In [ ]:
!pip install -q git+https://github.com/vblagoje/qdrant-haystack.git@haystack-v2 git+https://github.com/deepset-ai/haystack.git@openapi_container_v5

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 MB/s eta 0:00:00


In [ ]:
import getpass
import os
import json
import requests
from qdrant_haystack import QdrantDocumentStore
from qdrant_haystack.retriever import QdrantRetriever

from haystack import Pipeline
from haystack.components.generators.utils import default_streaming_callback
from haystack.components.writers import DocumentWriter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.converters import OpenAPIServiceToFunctions
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.generators.chat import GPTChatGenerator
from haystack.components.connectors import OpenAPIServiceConnector
from haystack.dataclasses import ChatMessage

## 2. Setting up Qdrant vector DB, create indexing pipeline

In [ ]:
document_store = QdrantDocumentStore(
    path="./qdrant_v1",
    index="Document",
    embedding_dim=768,
    recreate_index=True,
    hnsw_config={"m": 16, "ef_construct": 64}  # Optional
)

In [ ]:
indexing_pipeline = Pipeline()
indexing_pipeline.add_component("spec_to_functions", OpenAPIServiceToFunctions())
indexing_pipeline.add_component("embedder", SentenceTransformersDocumentEmbedder(model_name_or_path="sentence-transformers/all-mpnet-base-v2"))
indexing_pipeline.add_component("writer", DocumentWriter(document_store=document_store))
indexing_pipeline.connect(connect_from="spec_to_functions", connect_to="embedder")
indexing_pipeline.connect(connect_from="embedder", connect_to="writer")

## 3. Index two openapi specs (Github and SerperDev), along with system prompts

In [ ]:
indexing_pipeline.run(data={"sources":["https://t.ly/eBODl", "https://t.ly/Cn-Tv"],
                            "system_messages":[requests.get("https://t.ly/_1vOw").text,
                                               requests.get("https://t.ly/fZR09").text]})

In [ ]:
retrieval_pipeline = Pipeline()
retrieval_pipeline.add_component("embedder", SentenceTransformersTextEmbedder(model_name_or_path="sentence-transformers/all-mpnet-base-v2"))
retrieval_pipeline.add_component("retriever", QdrantRetriever(document_store=document_store))
retrieval_pipeline.connect(connect_from="embedder", connect_to="retriever")

## 4. API keys, set up simple authentication mechanism

In [ ]:
llm_api_key = getpass.getpass("Enter LLM provider api key:")


Enter LLM provider api key:··········


In [ ]:
serper_dev_key = getpass.getpass("Enter serperdev api key:")
services_auth = {"SerperDev":serper_dev_key}

Enter serperdev api key:··········


## 5. Retrieval step - service invocation

Based on the prompt, retrieve the right service spec from Qdrant, inject spec and system prompt into the pipeline

In [ ]:
invoke_service_pipe = Pipeline()
invoke_service_pipe.add_component("functions_llm", GPTChatGenerator(api_key=llm_api_key, model_name="gpt-3.5-turbo-0613"))
invoke_service_pipe.add_component("openapi_container", OpenAPIServiceConnector(services_auth))
invoke_service_pipe.connect("functions_llm.replies", "openapi_container.messages")

In [ ]:
user_prompt = "Search web and tell me why was Sam Altman ousted from OpenAI"
# user_prompt = "Compare branches main and openapi_container_v5, in project deepset-ai, repo haystack"

In [ ]:
top_k = 1
top_k_result = retrieval_pipeline.run(data={"text": user_prompt, "top_k": top_k})
top_k_docs = top_k_result["retriever"]["documents"][:top_k]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
top_1_document = top_k_docs[0]
openai_functions_definition = json.loads(top_1_document.content)
openapi_spec = top_1_document.meta["spec"]

service_response = invoke_service_pipe.run(data={"messages":[ChatMessage.from_user(user_prompt)],
                                                 "generation_kwargs": {"functions": [openai_functions_definition]},
                                                 "service_openapi_spec": openapi_spec})

## 6. Generate LLM response

Inject service response into LLM context, pair it with system prompt

In [ ]:
gen_pipe = Pipeline()
llm = GPTChatGenerator(api_key=llm_api_key, model_name="gpt-4-1106-preview", streaming_callback=default_streaming_callback)
gen_pipe.add_component("llm", llm)

github_pr_prompt_messages = [ChatMessage.from_system(top_1_document.meta["system_message"])] + service_response["openapi_container"]["service_response"]
final_result = gen_pipe.run(data={"messages": github_pr_prompt_messages})

Sam Altman was ousted from his position as CEO of OpenAI by the company's board. The decision was based on concerns that Altman was too focused on building OpenAI's business while not paying enough attention to the dangers of artificial intelligence. This move was surprising to many industry allies and rank-and-file employees who supported the charismatic founder. However, it has been reported that Altman was not consistently candid in his communications with the board, which also played a role in his firing.

After his ouster, there were discussions about bringing him back to OpenAI, but these efforts failed, and instead, Altman was hired by Microsoft just hours after OpenAI confirmed that he would not be returning as CEO.

## Thank you, questions?

<a href="www.qr-code-generator.com/" border="0" style="cursor:default" rel="nofollow"><img src="https://chart.googleapis.com/chart?cht=qr&chl=https%3A%2F%2Fgithub.com%2Fvblagoje%2Fnotebooks%2Fblob%2Fmain%2Fhaystack2x-demos%2Fhaystack_rag_services_demo.ipynb&chs=180x180&choe=UTF-8&chld=L|2"></a>

## Links:
- https://github.com/deepset-ai/haystack/
- https://haystack.deepset.ai/community
- https://haystack.deepset.ai/advent-of-haystack
- https://x.com/vladblagoje